In [ ]:
import json
import os
import pandas as pd

from util.config import versioned

df_path = versioned.get_path("matched.parquet")
df_all = None  # TODO Do I need it for visibility?
if os.path.exists(df_path):
    print(f"Reading memoized {df_path}")
    df_all = pd.read_parquet(df_path)
df_all = df_all[df_all["overall_score"] >= 11]

with open(versioned.get_path("people_krs.jsonl")) as pkrs:
    result = [json.loads(jline) for jline in pkrs]

In [ ]:
df = pd.DataFrame({'employed_end': df_all["employed_end"]})
df_all

In [ ]:
import matplotlib.pyplot as plt

# 2. Convert the 'event_date' column to datetime objects
df['employed_end'] = pd.to_datetime(df['employed_end'])

# 3. Extract the year and count the occurrences
yearly_counts = df['employed_end'].dt.year.value_counts()

# 4. Sort the counts by year (the index) for a chronological chart
yearly_counts = yearly_counts.sort_index()

# 5. Create the bar chart
plt.figure(figsize=(8, 5))
ax = yearly_counts.plot(
    kind='bar',
    color='coral',
    title='Number of Events per Year'
)

ax.set_xlabel("Year")
ax.set_ylabel("Number of Occurrences")
plt.xticks(rotation=0) # Keep year labels horizontal
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# 3. Extract the year and count the occurrences
yearly_counts = df_all['employed_krs'].value_counts()

# 4. Sort the counts by year (the index) for a chronological chart
yearly_counts = yearly_counts.sort_values()

# 5. Create the bar chart
plt.figure(figsize=(8, 5))
ax = yearly_counts.plot(
    kind='bar',
    color='coral',
    title='Number of Events per Year'
)

ax.set_xlabel("Year")
ax.set_ylabel("Number of Occurrences")
plt.xticks(rotation=0) # Keep year labels horizontal
plt.tight_layout()
plt.show()

yearly_counts